# Thermography

## Libraries

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, Dense
from keras.losses import binary_crossentropy
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

## Data selection

In [ ]:
class Data:
	def __init__(self, root):
		self.table = self.__generate_table(root)
		self.training, self.validation, self.test = None, None, None

	def __generate_table(self, root):
		files = []

		for category in os.listdir(root):
			for filename in os.listdir(root + category):
				files.append([root + category + "/" + filename, category])

		return pd.DataFrame(files, columns=["filename", "category"])

In [ ]:
data = Data("./data/thermography/")

In [ ]:
data.table.head(1)

## Preprocessing and transformation

In [ ]:
data.table.category = data.table.category.factorize()[0].astype(str)

In [ ]:
data.table.head(1)

In [ ]:
data.training, data.test = train_test_split(data.table, test_size=0.15, shuffle=True)

In [ ]:
print(f"Raw: {data.table.category.value_counts().values}")
print(f"Training: {data.training.category.value_counts().values}")
print(f"Test: {data.test.category.value_counts().values}")

In [ ]:
train_datagen = ImageDataGenerator(
	rescale=1./255,
	validation_split=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
	dataframe=data.training,
    x_col="filename",
    y_col="category",
    target_size=(215, 538),
    batch_size=25,
    class_mode="categorical",
    shuffle=True,
    color_mode="rgb",
    subset="training"
)

validation_generator = train_datagen.flow_from_dataframe(
	dataframe=data.training,
    x_col="filename",
    y_col="category",
    target_size=(215, 538),
    batch_size=25,
    class_mode="categorical",
    shuffle=True,
    color_mode="rgb",
    subset="validation"
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=data.test,
    x_col="filename",
    y_col="category",
    target_size=(215, 538),
    batch_size=1,
    class_mode="categorical",
    shuffle=False,
    color_mode="rgb",
)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3)

image, label = train_generator.next()

for i in range(3):    
    ax[i].imshow(image[i])
    ax[i].title.set_text(str(label[i]))
    ax[i].axis("off")
    
plt.show()

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(215, 538,  3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(GlobalMaxPooling2D())

model.add(Dropout(rate=0.4))
model.add(Dense(32, activation="relu"))
model.add(Dropout(rate=0.4))
model.add(Dense(2, activation="softmax"))

model.summary()

In [ ]:
model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=0.0005), metrics=["accuracy"])

In [ ]:
with open("./model.json", "w") as json_file:
	json_file.write(model.to_json())

In [ ]:
checkpoint = ModelCheckpoint("weights/weights-{epoch:02d}.hdf5", monitor="val_accuracy", verbose=1, save_best_only=False, mode="max")

In [ ]:
history = model.fit(train_generator, epochs=600, validation_data=validation_generator, callbacks=[checkpoint])